# npyblast baseline

For the first baselines we want to generate accuracy, precision, recall and F1 score for each of the EC numbers.

The first dataset is the price dataset.

Requires: npysearch, sciutil

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
from sciutil import SciUtil
u = SciUtil()

base_dir = '/disk1/ariane/pycharm/CARE/'

# Input a ranked ordered list of EC numbers from best to worst.

# I'll 
    
def compute_accuracy_baseline1(predicted_ecs, true_ecs):
    pred_level1 = np.zeros(len(predicted_ecs))
    pred_level2 = np.zeros(len(predicted_ecs))
    pred_level3 = np.zeros(len(predicted_ecs))
    pred_level4 = np.zeros(len(predicted_ecs))

    # Basically given we have a highly multiclass problem doing precision recall probably doesn't make sense
    for i, predicted_ec in enumerate(predicted_ecs):
        true_ec = true_ecs[i]
        # First check level 1
        pred1 = predicted_ec.split('.')[0]
        true_ecs1 = [ec.split('.')[0] for ec in true_ec.split(';')]
        if pred1 in true_ecs1:
            pred_level1[i] = 1

        # Do the same for each other level
        pred2 = predicted_ec.split('.')[1]
        true_ecs2 = [ec.split('.')[1] for ec in true_ec.split(';')]
        if pred2 in true_ecs2:
            # Check also that pred1 was correct
            if pred_level1[i] == 1:
                pred_level2[i] = 1
        
        # Do the same for each other level
        pred3 = predicted_ec.split('.')[2]
        true_ecs3 = [ec.split('.')[2] for ec in true_ec.split(';')]
        if pred3 in true_ecs3:
            # Check previous levels were correct
            if pred_level1[i] == 1 and pred_level2[i] == 1:
                pred_level3[i] = 1
        
        # Do the same for each other level
        pred4 = predicted_ec.split('.')[3]
        true_ecs4 = [ec.split('.')[3] for ec in true_ec.split(';')]
        if pred4 in true_ecs4:
            if pred_level1[i] == 1 and pred_level2[i] == 1 and pred_level3[i] == 1:
                pred_level4[i] = 1
    # Print out the accuracy
    u.dp(['Acc level 1:', round(np.mean(pred_level1)*100, 2), 
          '\nAcc level 2:', round(np.mean(pred_level2)*100, 2), 
          '\nAcc level 3:', round(np.mean(pred_level3)*100, 2), 
          '\nAcc level 4:', round(np.mean(pred_level4)*100, 2)])
    
    return np.mean(pred_level1), np.mean(pred_level2), np.mean(pred_level3), np.mean(pred_level4)


def make_default_training_fasta():
    swissprot = pd.read_csv(f'{base_dir}processed_data/protein2EC.csv')
    train_indices = np.loadtxt(f'{base_dir}splits/task1/protein2EC_train_indices.txt', dtype=int)
    swissprot_train = swissprot.iloc[train_indices]
    entries = swissprot_train['Entry'].values
    sequences = swissprot_train['Sequence'].values
    #save sequences to fasta
    with open(f'{base_dir}protein2EC_train.fasta', 'w') as f:
        for entry, sequence in zip(entries, sequences):
            f.write('>{}\n{}\n'.format(entry, sequence))
    u.dp(['Default training dataset built using: ', f'{base_dir}processed_data/protein2EC.csv', 
          '\nIncides:', f'{base_dir}splits/task1/protein2EC_train_indices.txt', 
          '\nOutput:', f'{base_dir}protein2EC_train.fasta'])

def make_default_price_fasta():
    df = pd.read_csv(f'{base_dir}splits/task1/price_protein_test.csv', sep='\t')
    #write seqs to fasta format
    with open(f'{base_dir}splits/task1/price_protein_test.fasta', 'w') as f:
        for i, (entry, seq) in enumerate(zip(entries, seqs)):
            f.write('>{}\n{}\n'.format(entry, seq))
    u.dp(['Default price dataset built using: ', f'{base_dir}splits/task1/price_protein_test.csv', 
      '\nOutput:', f'{base_dir}splits/task1/price_protein_test.fasta'])

def get_uniprot2ec():
    swissprot = pd.read_csv(f'{base_dir}processed_data/protein2EC.csv')
    id2ec = swissprot.set_index('Entry')['EC number'].to_dict()
    return id2ec

def get_price2ec():
    df = pd.read_csv(f'{base_dir}splits/task1/price_protein_test.csv')
    id2ec = df.set_index('Entry')['EC number'].to_dict()
    return id2ec
    
def get_default_training_fasta_path():
    return f'{base_dir}processed_data/protein2EC_train.fasta'
    
def get_default_price_fasta_path():
    return f'{base_dir}processed_data/price_protein_test.fasta'

def get_validation30():
    return f'{base_dir}processed_data/validation_30.fasta'

def get_validation50():
    return f'{base_dir}processed_data/validation_50.fasta'
    
def get_validation70():
    return f'{base_dir}processed_data/validation_70.fasta'

def get_validation90():
    return f'{base_dir}processed_data/validation_90.fasta'

def get_promisc():
    return f'{base_dir}processed_data/promiscuous.fasta'

## Make the default datasets

In [3]:
# make_default_training_fasta()
# make_default_price_fasta()

## Perform npysearch

In [4]:
import npysearch as npy

# Lets also look at the protein our query is the query genome and our database is going to be ecoli.
results_prot = npy.blast(query=get_default_price_fasta_path(),
                         database=get_default_training_fasta_path(),
                         minIdentity=0.1,
                         maxAccepts=1,
                         alphabet="protein")
results = pd.DataFrame(results_prot)  # Convert this into a dataframe so that we can see it more easily


   Read database: 100.0% (57 MB)                    
Analyze database: 100.0% (168k)                    
  Index database: 100.0% (168k)                    
    Read queries: 100.0% (58 kB)                      
 Search database: 100.0% (148.0)                    


## Map the targetID which is the prediction to the the predicted EC number

In [5]:
results['predicted_ecs'] = results['TargetId'].map(get_uniprot2ec())
results['true_ecs'] = results['QueryId'].map(get_price2ec())
results

,QueryId,TargetId,QueryMatchStart,QueryMatchEnd,TargetMatchStart,TargetMatchEnd,QueryMatchSeq,TargetMatchSeq,NumColumns,NumMatches,NumMismatches,NumGaps,Identity,Alignment,predicted_ecs,true_ecs
0,NP_384124,P42604,4,501,1,495,PSSILLSPDDNVVVATAAIAPGDRLAGGVSAVARIEPGHKAAIRRI...,MQYIKIHALDNVAVALADLAEGTEVSVDNQTVTLRQDVARGHKFAL...,502,171,320,11,0.341,3X1=5X3=1X2=1X1=2X1=1X1=9X1=1X1D1=2D3X3=1X1=3X...,4.2.1.7,4.2.1.5
1,NP_384306,Q9PFE2,169,294,1,141,HQGSGVTDAVLAEFNDDPRDIVDFARLAKPGEFGRYAPRVFEFAER...,MITITLPDGSRREFEGPVSVMQVAHAIGPGLAKATIAGQIDGGHLV...,142,23,102,17,0.162,7X1=4X2=1I2X1=6X1=4X2=16X1=4X4D2X1=8X4=2X11D1=...,6.1.1.3,2.7.1.59
2,NP_384741,A0AXY5,1,292,269,546,MMMKTVAVIDIGKTNAKVALVDLERFEEIAVRKSGNGVSDDGPYPH...,GILKTVAVKAPGFGDRRKALLEDIAILTGGQVIAEETGLTLEKTTL...,292,47,231,14,0.161,3X5=3X1=6X2=2X5I1X2=4X1=6X3I1X2I3X1I1=4X1=2X1=...,5.6.1.7,2.7.1.5
3,WP_010211220,Q1JUQ1,1,578,1,583,MPDKKPGLRSAQWFGTADKNGFMYRSWMKNQGIADHQFHGKPIIGI...,MSATKPRLRSTQWFGTNDKNGFMYRSWMKNQGIPDHEFDGRPIIGI...,584,459,118,7,0.786,1=3X2=1X3=1X5=1X16=1X2=1X1=1X1=1X20=2X7=2X21=1...,4.2.1.25,4.2.1.25
4,WP_010211217,Q6FFQ0,1,524,1,526,MTSFLGHNYIGGQRSANGSVTLQSVDATSGEALPQHFYQATPQEVD...,MSENNGKQFINGQRVAANAPTIESINATDYQPTGYLFSQATLDEVD...,526,304,220,2,0.578,1=4X1=3X1=1X3=1X1=4X1=2X1=2X2=8X1=1X3=2X3=1X2=...,1.2.1.26,1.2.1.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,NP_384884,Q7VUQ3,1,261,88,357,MIVHFEPASEEGIASVVRSAAAERVTLAVVGGGTRAGLGNPVRADR...,RLADIRPDVVLGMGGYVAFPGGVMAALRRTPLVVHEQNAVAGTANR...,276,48,207,21,0.174,6X1=4X1=4X1=9X1=16X1=1X1=1X1=1X1I1=9X1=9X1=1X2...,2.4.1.227,1.1.3.15
144,NP_384885,P52074,24,438,1,407,ADPHVAESETILRKCVHCGFCTATCPTYVVLGDELDSPRGRIYLIK...,MQTQLTEEMRQNARALEADSILRACVHCGFCTATCPTYQLLGDELD...,419,122,281,16,0.291,6X1=17X1=1I5X1I5X1=2I3X1=2X1=1X1=3X1=1X3I1X1I3...,1.1.99.14,1.1.3.15
145,NP_384521,Q652Q8,1,314,17,329,MSSARKIIIDTDPGQDDAAAIMLALGSPEEIEVLGITAVAGNVPLT...,PPTEEKVIIDTDPGIDDSVAIMMAFEAPGVKVVGLTTIFGNCTTSH...,315,104,208,3,0.330,5X1=1X7=1X2=2X3=1X1=3X1=1X1I2X1=1X1=1X1=3X2=7X...,3.2.2.3,3.2.2.3
146,NP_384733,B1IB49,81,430,1,363,LEDCAVIQQLTRATPAVSLHIPWDKVSDLGALKEKGSALGLSFDAM...,MQYSEIMIRYGELSTKGKNRMRFINKLRNNISDVLSIYAQVKVTAD...,363,42,308,13,0.116,44X1=10X1=8X2=18X2=2X1=13X1=2X1=3X1=3X1D2X2=24...,2.8.1.4,5.3.1.14


## Compute accuracy for the prediction vs the true values for each level

In [6]:
u.dp(['Price dataset'])
compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)


--------------------------------------------------------------------------------
                                 Price dataset	                                 
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Acc level 1:	73.65	
Acc level 2:	70.27	
Acc level 3:	61.49	
Acc level 4:	35.14	 
--------------------------------------------------------------------------------


(0.7364864864864865,
 0.7027027027027027,
 0.6148648648648649,
 0.35135135135135137)

## Do the same for each of the percentage splits

In [7]:
import npysearch as npy

# Lets also look at the protein our query is the query genome and our database is going to be ecoli.
results_prot = npy.blast(query=get_validation30(),
                         database=get_default_training_fasta_path(),
                         minIdentity=0.1,
                         maxAccepts=1,
                         alphabet="protein")
results = pd.DataFrame(results_prot)  # Convert this into a dataframe so that we can see it more easily

results['predicted_ecs'] = results['TargetId'].map(get_uniprot2ec())
results['true_ecs'] = results['QueryId'].map(get_uniprot2ec())
u.dp(['30% dataset'])

compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)


   Read database: 100.0% (57 MB)                    
Analyze database: 100.0% (168k)                    
  Index database: 100.0% (168k)                    
    Read queries: 100.0% (59 kB)                      
 Search database: 100.0% (175.0)                    


--------------------------------------------------------------------------------
                                  30% dataset	                                  
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Acc level 1:	62.64	
Acc level 2:	55.17	
Acc level 3:	52.87	
Acc level 4:	49.43	 
--------------------------------------------------------------------------------


(0.6264367816091954,
 0.5517241379310345,
 0.5287356321839081,
 0.4942528735632184)

In [8]:
import npysearch as npy

# Lets also look at the protein our query is the query genome and our database is going to be ecoli.
results_prot = npy.blast(query=get_validation50(),
                         database=get_default_training_fasta_path(),
                         minIdentity=0.1,
                         maxAccepts=1,
                         alphabet="protein")
results = pd.DataFrame(results_prot)  # Convert this into a dataframe so that we can see it more easily

results['predicted_ecs'] = results['TargetId'].map(get_uniprot2ec())
results['true_ecs'] = results['QueryId'].map(get_uniprot2ec())
u.dp(['50% dataset'])

compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)


   Read database: 100.0% (57 MB)                    
Analyze database: 100.0% (168k)                    
  Index database: 100.0% (168k)                    
    Read queries: 100.0% (74 kB)                      
 Search database: 100.0% (196.0)                    


--------------------------------------------------------------------------------
                                  50% dataset	                                  
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Acc level 1:	93.88	
Acc level 2:	90.31	
Acc level 3:	87.24	
Acc level 4:	84.18	 
--------------------------------------------------------------------------------


(0.9387755102040817,
 0.9030612244897959,
 0.8724489795918368,
 0.8418367346938775)

In [9]:

# Lets also look at the protein our query is the query genome and our database is going to be ecoli.
results_prot = npy.blast(query=get_validation70(),
                         database=get_default_training_fasta_path(),
                         minIdentity=0.1,
                         maxAccepts=1,
                         alphabet="protein")
results = pd.DataFrame(results_prot)  # Convert this into a dataframe so that we can see it more easily
u.dp(['30-70% dataset'])

results['predicted_ecs'] = results['TargetId'].map(get_uniprot2ec())
results['true_ecs'] = results['QueryId'].map(get_uniprot2ec())
compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)


   Read database: 100.0% (57 MB)                    
Analyze database: 100.0% (168k)                    
  Index database: 100.0% (168k)                    
    Read queries: 100.0% (74 kB)                      
 Search database: 100.0% (204.0)                    


--------------------------------------------------------------------------------
                                30-70% dataset	                                 
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Acc level 1:	99.02	
Acc level 2:	96.08	
Acc level 3:	94.61	
Acc level 4:	89.71	 
--------------------------------------------------------------------------------


(0.9901960784313726, 0.9607843137254902, 0.946078431372549, 0.8970588235294118)

In [10]:

# Lets also look at the protein our query is the query genome and our database is going to be ecoli.
results_prot = npy.blast(query=get_validation90(),
                         database=get_default_training_fasta_path(),
                         minIdentity=0.1,
                         maxAccepts=1,
                         alphabet="protein")
results = pd.DataFrame(results_prot)  # Convert this into a dataframe so that we can see it more easily
u.dp(['70-90% dataset'])

results['predicted_ecs'] = results['TargetId'].map(get_uniprot2ec())
results['true_ecs'] = results['QueryId'].map(get_uniprot2ec())
compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)


   Read database: 100.0% (57 MB)                    
Analyze database: 100.0% (168k)                    
  Index database: 100.0% (168k)                    
    Read queries: 100.0% (75 kB)                      
 Search database: 100.0% (206.0)                    


--------------------------------------------------------------------------------
                                70-90% dataset	                                 
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Acc level 1:	99.03	
Acc level 2:	99.03	
Acc level 3:	99.03	
Acc level 4:	97.57	 
--------------------------------------------------------------------------------


(0.9902912621359223,
 0.9902912621359223,
 0.9902912621359223,
 0.9757281553398058)

In [11]:

# Lets also look at the protein our query is the query genome and our database is going to be ecoli.
results_prot = npy.blast(query=get_promisc(),
                         database=get_default_training_fasta_path(),
                         minIdentity=0.1,
                         maxAccepts=1,
                         alphabet="protein")
results = pd.DataFrame(results_prot)  # Convert this into a dataframe so that we can see it more easily
u.dp(['Promisuous dataset'])

results['predicted_ecs'] = results['TargetId'].map(get_uniprot2ec())
results['true_ecs'] = results['QueryId'].map(get_uniprot2ec())
compute_accuracy_baseline1(results['predicted_ecs'].values, results['true_ecs'].values)


   Read database: 100.0% (57 MB)                    
Analyze database: 100.0% (168k)                    
  Index database: 100.0% (168k)                    
    Read queries: 100.0% (71 kB)                      
 Search database: 100.0% (179.0)                    


--------------------------------------------------------------------------------
                              Promisuous dataset	                               
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
 Acc level 1:	90.5	
Acc level 2:	86.03	
Acc level 3:	84.36	
Acc level 4:	83.24	 
--------------------------------------------------------------------------------


(0.9050279329608939,
 0.8603351955307262,
 0.8435754189944135,
 0.8324022346368715)